# Simulation

Zhentao Shi


In [ ]:
library(magrittr)
library(tibble)
library(plyr)
set.seed(888)

## Simulation


* check finite-sample performance
* bootstrap, a data-driven inference procedure
* generate non-standard distributions
* approximate integrals with no analytic expressions


## Appetizer

calculating $\pi$.

In [ ]:
require(plotrix)
require(grid)

plot(c(-1, 1), c(-1, 1), type = "n", asp = 1, xlab = "x", ylab = "y")
rect(-1, -1, 1, 1)
draw.circle(0, 0, 1)
points(x = runif(100) * 2 - 1, y = runif(100) * 2 - 1, pch = 20, col = "blue")

By laws of large numbers, $\pi$ can be approximated by a stochastic algorithm

$$
E\left[\boldsymbol{1}\left\{ x^{2}+y^{2}\leq1\right\} \right] = \frac{\pi r^{2}}{\left(2r\right)^{2}}= \frac{\pi}{4}
$$

it implies  $\pi=4\times E[ 1 \{  x^{2}+y^{2}\leq1 \}]$. 

In [ ]:
n <- 20000000
Z <- 2 * matrix(runif(n), ncol = 2) - 1 # uniform distribution in [-1, 1]

inside <- mean(sqrt(rowSums(Z^2)) <= 1) # the center of the circle is (0, 0)
cat("The estimated pi = ", inside * 4, "\n")


* Sample size can be made as large as the computer's memory permits.
* Iterate it with average of averages and so on, for higher accuracy.


## Writing Script

* A script is a piece of code for a particular purpose. Thousands of lines are not written from the beginning to the end. 

* Recursively development
  * small manageable tasks
  * test code constantly 
  * encapsulate into DIY functions
  

* Integrate small pieces into the super structure. 
* Add comments to the script to facilitate readability


## Finite Sample Evaluation

* Real world sample is finite
* Asymptotic theory is a mathematical apparatus to approximate finite sample distributions
* Modern econometric theory is built on asymptotics
* Simulation is one way to evaluate the accuracy of approximation.

## Example

For the OLS estimator, 

* Classical views $X$ as fixed regressions and only cares about the randomness of the error term.
* Modern econometrics textbook emphasizes that a random $X$ is more appropriate
for econometrics applications. 
* In rigorous textbooks, the moment of $X$ is explicitly stated as $E[X_i X_i'] < \infty$.

* A Pareto distribution with shape coefficient between 1 and 2 has finite population mean, but infinite variance. 
* If $X$ follows a
[Pareto distribution](https://en.wikipedia.org/wiki/Pareto_distribution) with shape coefficient 1.5, it violates the assumptions for OLS stated in most of econometric textbooks.
* Question: Is asymptotic inferential theory for the OLS estimator valid? 

We write a script to investigate this problem. The following steps develop the code.

 1. given a sample size, get the OLS `b_hat` and its associated `t_value`.
 2. wrap `t_value` as a user-defined function so that we can reuse it for many times.
 3. given a sample size, report the size under two distributions.
 4. wrap step 3 again as a user-defined function, ready for different sample sizes.
 5. develop the super structure to connect the workhorse functions.
 6. add comments and documentation.


### $t$-statistic

Under the null:
the $k$-th element of the vector coefficient conditional on $X$ is
$$
\frac{\hat{\beta}_k - \beta_{k0} } {\mathrm{s.d.}(\hat{\beta}_k)}
\stackrel{a}{\to} 
 N\left(0 ,1\right).
$$

In [ ]:
# the workhorse functions
simulation <- function(n, type = "Normal", df = df) {
  # a function gives the t-value under the null
  if (type == "Normal") {
    e <- rnorm(n)
  } else if (type == "T") {
    e <- rt(n, df)
  }

  X <- cbind(1, VGAM::rpareto(n, shape = 1.5))
  Y <- X %*% b0 + e
  rm(e)

  bhat <- solve(t(X) %*% X, t(X) %*% Y)
  bhat2 <- bhat[2] # parameter we want to test

  e_hat <- Y - X %*% bhat
  sigma_hat_square <- sum(e_hat^2) / (n - 2)
  sig_B <- solve(t(X) %*% X) * sigma_hat_square
  t_value_2 <- (bhat2 - b0[2]) / sqrt(sig_B[2, 2])

  return(t_value_2)
}


In [ ]:
# report the empirical test size
report <- function(n) {
  # collect the test size from the two distributions
  # this function contains some repetitive code, but is OK for such a simple one
  TEST_SIZE <- rep(0, 3)

  # e ~ normal distribution, under which the t-dist is exact
  Res <- plyr::ldply(.data = 1:Rep, .fun = function(i) simulation(n, "Normal"))
  TEST_SIZE[1] <- mean(abs(Res) > qt(.975, n - 2))
  TEST_SIZE[2] <- mean(abs(Res) > qnorm(.975))

  # e ~ t-distribution, under which the exact distribution is complicated.
  # we rely on asymptotic normal distribution for inference instead
  Res <- plyr::ldply(.data = 1:Rep, .fun = function(i) simulation(n, "T", df))
  TEST_SIZE[3] <- mean(abs(Res) > qnorm(.975))

  return(TEST_SIZE)
}


In [ ]:
## the super structure
# set the parameters
Rep <- 1000
b0 <- matrix(1, nrow = 2)
df <- 1 # t dist. with df = 1 is Cauchy

# run the calculation of the empirical sizes for different sample sizes
NN <- c(5, 10, 200, 5000)
RES <- plyr::ldply(.data = NN, .fun = report)
names(RES) <- c("exact", "normal.asym", "cauchy.asym") # to make the results readable
RES$n <- NN
RES <- RES[, c(4, 1:3)] # beautify the print
print(RES)


### Simulation Results

* 1st column: the error is normal, use exact distribution theory to find the critical value (according to $t$-distribution.)

* 2nd column: still uses the normal error
  * change the critical value to asymptotic normal distribution
  
* 3rd column: error distribution is Cauchy
  * asymptotic approximation breaks down
  * test size does not converge to the nominal 5%  

### Observations

* $X$ is always Pareto 
* distribution of $X$ doesn't matter in our simulations

### Justification


The $t$-statistic

$$
T_{k}  =\frac{\widehat{\beta}_{k}-\beta_{k}}{\sqrt{s^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}} =\frac{\widehat{\beta}_{k}-\beta_{k}}{\sqrt{\sigma^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}}\cdot\frac{\sqrt{\sigma^{2}}}{\sqrt{s^{2}}}\\
  =\frac{\left(\widehat{\beta}_{k}-\beta_{k}\right)/\sqrt{\sigma^{2}\left[\left(X'X\right)^{-1}\right]_{kk}}}{\sqrt{\frac{e'}{\sigma}M_{X}\frac{e}{\sigma}/\left(n-K\right)}}.
$$

* Self-normalized $t$ statistic does not break down despite that $X'X/n$ does not converge
* Regardless the distribution of $X$, when the error term is normal 
  * numerator follows $N(0,1)$
  * demonimator follows $\chi^2$

### World View


* Fundamental question: how to quantify uncertainty.
  * data $(X_1,X_2,\ldots,X_n)$
  * sample mean $\bar{X}$
  * sample variance $s$
  * frequentist confidence interval about the population mean

* Asymptotics is imaginary.
* Let's be realistic: we have a finite sample $n$.

## Bootstrap

* Let $X_1, X_2, \ldots, X_n \sim F$ be an i.i.d. sample of $n$ observations following a distribution $F$. 
* The finite sample distribution of a statistic $T_n(\theta)\sim G_n(\cdot, F)$ usually depends on the sample size $n$, as well as the unknown true distribution $F$. 


### Key Idea

* Bootstrap replaces the unknown distribution $F$ in $G_n(\cdot, F)$ by the empirical distribution function

$$
\hat{F}_n(\cdot) = n^{-1} \sum_{i=1}^n 1\{\cdot \leq X_i\}
$$

* Bootstrap inference is drawn from the bootstrap distribution

$$
G_n(\cdot, \hat{F}_n)
$$



### Compare to Asymptotic Theory

* Bootstrap is a finite-sample practice 
* It doesn't refer to an imaginary world where $n\to \infty$ at its face value


* Asymptotic theory approximates $G_n(\cdot, F)$ by its limit 

$$G(\cdot, F) := \lim_{n\to\infty} G_n(\cdot, F).$$ 

* In many cases $G(\cdot, F)$ is independent of $F$ and it becomes $G(\cdot)$. Such a $T_n(\theta)$ is called *asymptotically pivotal*, 

In [ ]:
runif(10) %>%
  ecdf() %>%
  plot(, xlim = c(0, 1), main = "ECDF for uniform distribution")


### Nonparametric Bootstrap

* Implementation of bootstrap is a simulation exercise. 
* In an i.i.d. environment, $n$ observations are drawn with equal weight and **with replacement** from the realized sample

### Variants of Bootstrap Schemes

* Block bootstrap: preserve dependence structure
  * dependent dataset such as time series
  * clustering data or networks

* parametric bootstrap
  * In regressions we fix the regressors

* wild bootstrap: [[Davidson, 2010]](https://www.tandfonline.com/doi/abs/10.1198/jbes.2009.07221?journalCode=ubes20)
  * Heteroskedascity

In [ ]:
n <- 9 # real sample size
real_sample <- rnorm(n) # the real sample
d0 <- tibble(no = 1:n, x = real_sample)
print(d0)

###### bootstrap 
boot_Rep <- 3 # bootstrap 3 times
d_boot <- list() # save the bootstrap sample
for (b in 1:boot_Rep) {
  boot_index <- sample(1:n, n, replace = TRUE)
  d_boot[[b]] <- tibble(no = boot_index, x = real_sample[boot_index])
}

d_boot %>% as_tibble(, .name_repair = "minimal") %>% print()

### Bootstrap Estimation


* R package [boot](http://cran.r-project.org/web/packages/boot/index.html) provides a general function `boot()`.
* `ply`-family functions for repeated simulations. 

* Bootstrap is convenient. 
  * Analytic formula of the variance of an econometric estimator can be complex to derive or code up.

## Example

* One of the most popular estimators for a sample selection model is @heckman1977sample's two-step method

* Outcome equation

$$
y_i = x_i \beta + u_i
$$

and the selection equation be

$$
D_i = z_i \gamma + v_i
$$

* To obtain a point estimator, we simply run a Probit in the selection model, predict the probability of participation, and then run an OLS of $y_i$ on $x_i$ and $\lambda (\hat{D}_i)$ in the outcome model, where $\lambda(\cdot)$ is the inverse Mill's ratio. 

* From Heckman (1979)'s original paper, the asymptotic variance expression of the two-step estimator is very complicated.

* Instead of following the analytic formula, we can simply bootstrap the variance.

In [ ]:
# the dataset comes from
# Greene( 2003 ): example 22.8, page 786
library(sampleSelection)
data(Mroz87)
# equations
selection_eq <- lfp ~ -1 + age + faminc + exper + educ
outcome_eq <- wage ~ exper + educ

# Heckman two-step estimation
heck <- sampleSelection::heckit(selection_eq, outcome_eq, data = Mroz87)
print(lmtest::coeftest(heck))


* Below is the function for a single bootstrap. 

In [ ]:
n <- nrow(Mroz87)
boot_heck <- function() {
  indices <- sample(1:n, n, replace = T) # resample the index set
  Mroz87_b <- Mroz87[indices, ] # generate the bootstrap sample
  heck_b <- sampleSelection::heckit(selection_eq, outcome_eq, data = Mroz87_b)
  return(coef(heck_b))
}

* Implementation is just a repeated evaluation.

In [ ]:
# repeat the bootstrap
boot_Rep <- 199
Heck_B <- plyr::ldply(.data = 1:boot_Rep, .fun = function(i) boot_heck())

# collect the bootstrap outcomes
Heck_b_sd <- apply(Heck_B, 2, sd)[1:7]
print(Heck_b_sd)

* The standard errors from the analytical expression and those from bootstrap are comparable.
* The bootstrap estimates can also be used to directly compute the confidence intervals.



## Numerical Methods

* Numerical differentiation and integration
* Nothing to do with economics or econometrics

The partial derivative of a multivariate function
$f:R^K \mapsto R$ at a point $x_0 \in R^K$ is

$$
\frac{\partial f(x)}{\partial x_k}\bigg|_{x=x_0}=\lim_{\epsilon \to 0}
\frac{f(x_0+\epsilon \cdot e_k) - f(x_0 - \epsilon \cdot e_k)}{2\epsilon},
$$

where $e_k = (0,\ldots,0,1,0,\ldots,0)$ is the identifier of the $k$-th coordinate.

* Numerical execution in a computer follows the basic definition to evaluate
$f(x_0\pm\epsilon \cdot e_k))$ with a small
$\epsilon$. 

* How small is small? Usually we try a sequence of $\epsilon$'s until
the numerical derivative is stable. 

* There are also more sophisticated algorithms.

## Caution

* Numerical methods are not panacea
* Not all functions are differentiable or integrable.
* Before turning to numerical methods, it is always imperative to try to understand the behavior of the function at the first place.
* Some symbolic software, such as `Mathematica` or `Wolfram Alpha`, is a useful tool for this purpose. 
* R is weak in symbolic calculation despite the existence of a few packages for this purpose.

## Stochastic Methods

* An alternative to numerical integration is the stochastic methods.
* The underlying principle of stochastic integration is the law of large numbers.

* Let  $\int h(x) d F(x)$ be an integral where $F(x)$ is a probability distribution.
* Approximate the integral by
$\int h(x) d F(x) \approx S^{-1} \sum_{s=1}^S h(x_s)$, where $x_s$ is randomly
generated from $F(x)$.

* When $S$ is large, a law of large numbers gives

$$
S^{-1} \sum_{s=1}^S h(x_s) \stackrel{\mathrm{p}}{\to} E[h(x)] = \int h(x) d F(x).
$$

* If the integration is carried out not in the entire support of $F(x)$ but on a subset $A$, then

$$
\int_A h(x) d F(x) \approx S^{-1} \sum_{s=1}^S h(x_s) \cdot 1\{x_s \in A\},
$$

where $1\{\cdot\}$ is the indicator function.

* In theory, use an $S$ as large as possible.
* In reality, constrained by the computer's memory and computing time.
* No clear guidance of the size of $S$ in practice. 
* Preliminary experiment can help decide an $S$ that produces stable results.

* Stochastic integration is popular in econometrics and statistics, thanks to its convenience in execution.

### Random Variable Generation

* If the CDF $F(X)$ is known, we can generate random variables that follow such a distribution.
  * Draw $U$ from  $\mathrm{Uniform}(0,1)$
  * Compute $X = F^{-1}(U)$

## Markov Chain Monte Carlo

* If the pdf $f(X)$ is known, we can generate a sample by *importance sampling*
  * [Metropolis-Hastings algorithm](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm)
 (MH algorithm) is such a method.
  * MH is one of the [Markov Chain Monte Carlo](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) methods.
* It can be implemented in the R package `mcmc`.
  * [This page](https://chi-feng.github.io/mcmc-demo/) contains demonstrative
examples of MCMC.

### Metropolis-Hastings Algorithm

* Theory of the MH requires long derivation
* Implementation is straightforward.

### Example

Use MH to generate a sample of normally distributed observations with
$\mu = 1$ and $\sigma = 0.5$.

* In the function `metrop`, we provide the logarithm of the density of

$$
\log f(x) = -\frac{1}{2} \log (2\pi) - \log \sigma - \frac{1}{2\sigma^2} (x-\mu)^2
$$

  * The first term can be omitted as it is irrelevant to the parameter.

In [ ]:
h <- function(x, mu = 1, sd = 0.5) {
  y <- -log(sd) - (x - mu)^2 / (2 * sd^2)
  # y <- log( dnorm(x, 1, 0.5) )
} # un-normalized function (doesn't need to integrate as 1)

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 100, nspac = 1)

par(mfrow = c(3, 1))
par(mar = c(2, 2, 1, 1))
plot(out$batch, type = "l") # a time series with flat steps

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 100, nspac = 10)
plot(out$batch, type = "l") # a time series looks like a white noise

out <- mcmc::metrop(obj = h, initial = 0, nbatch = 10000, nspac = 10)
plot(density(out$batch), main = "", lwd = 2)

xbase <- seq(-2, 2, 0.01)
ynorm <- dnorm(xbase, mean = 1, sd = 0.5)
lines(x = xbase, y = ynorm, type = "l", col = "red")


### Outcomes

* The first panel is a time series where
the marginal distribution of each observations follows $N(1,0.5^2)$. 
  * Time dependence is visible
  * flat regions are observed when the Markov chain rejects a new proposal
so the value does not update over two periods. 

* The middle panel collects the time series every 10 observations on the Markov chain
  * serial correlation is weakened
  * No flat region is observed
 
  
* The third panel compares     
  * kernel density of the simulated observations (black curve) 
  * density function of $N(1,0.5^2)$ (red curve).


### Bayesian Inference


* Bayesian framework offers a coherent and natural language for statistical decision. 
* Bayesian views both the data $\mathbf{X}_{n}$ and the
parameter $\theta$ as random variables
* Before observeing the data, researcher holds a *prior distribution* $\pi$ about $\theta$
* After observing the data, researcher updates the prior distribution to a *posterior distribution* $p(\theta|\mathbf{X}_{n})$. 


### Bayes Theorem

* Let $f(\mathbf{X}_{n}|\theta)$ be the likelihood
* Let $\pi$ be the prior

* The celebrated Bayes Theorem is

$$
p(\theta|\mathbf{X}_{n})\propto f(\mathbf{X}_{n}|\theta)\pi(\theta)
$$




### Classical Analytical Example 

* $\mathbf{X}_{n}=(X_{1},\ldots,X_{n})$ is an iid sample drawn from a normal distribution with unknown $\theta$ and known $\sigma$
* If a researcher's prior distribution
$\theta\sim N(\theta_{0},\sigma_{0}^{2})$, her posterior distribution
is, by some routine calculation, also a normal distribution

$$
p(\theta|\mathbf{x}_{n})\sim N\left(\tilde{\theta},\tilde{\sigma}^{2}\right),
$$

where
$\tilde{\theta}=\frac{\sigma^{2}}{n\sigma_{0}^{2}+\sigma^{2}}\theta_{0}+\frac{n\sigma_{0}^{2}}{n\sigma_{0}^{2}+\sigma^{2}}\bar{x}$
and
$\tilde{\sigma}^{2}=\frac{\sigma_{0}^{2}\sigma^{2}}{n\sigma_{0}^{2}+\sigma^{2}}$.


### Bayesian Credible Set

$$
\left(\tilde{\theta}-z_{1-\alpha/2}\cdot\tilde{\sigma},\ \tilde{\theta}+z_{1-\alpha/2}\cdot\tilde{\sigma}\right).
$$

* Posterior distribution depends on $\theta_{0}$ and $\sigma_{0}^{2}$
from the prior. 
* When the sample size is sufficiently, the data overwhelms the prior


### Frequentist Confidence Internval


* $\hat{\theta}=\bar{x}\sim N(\theta,\sigma^{2}/n)$. 

* Confidence interval is

$$
\left(\bar{x}-z_{1-\alpha/2}\cdot\sigma/\sqrt{n},\ \bar{x}-z_{1-\alpha/2}\cdot\sigma/\sqrt{n}\right).
$$

### Comparison

* Bayesian produces a posterior distribution
  * The posterior distribution implies point estimates and credible set
  * Data are fixed (invariant)
  * A prior distribution is needed

* Frequestist produces a point estimator
  * Before data are observed, the point estimator is random
  * Inference is imply by the point estimator **before observation**
  * Only data are realized, the point estimate is a fixed number
  * No prior distribution is needed
  

### Code Example

* Prior distribution $\mu \sim Beta(2,5)$
* [Beta distribution](https://en.wikipedia.org/wiki/Beta_distribution) is flexible

In [ ]:
# prior distribution
xbase = seq(0.01, 0.99, by = 0.01)
plot( x = xbase, y = dbeta(xbase, 2, 5), type = "l" )


* The nature generates data

In [ ]:
n <- 20
x <- rnorm(n) + 0.9
print(sort(x))

* The researcher correctly specifies the normal model
* Given the data, she infers $\theta$ according to the posterior distribution

In [ ]:
loglik <- function(theta) {   y <- sum( log( dnorm( x, theta, 1 ) ) ) } 

posterior <- function(theta) { loglik(theta) + log(dbeta(theta, 2, 2))  }

nbatch <- 10000
out <- mcmc::metrop(obj = posterior, initial = 0.1, nbatch = nbatch, nspac = 10)$batch
out <- out[-(1:round(nbatch/10))] # remove the burn-in period

plot(density(out))

print( quantile(out, probs = c(0.025, 0.975) ))

### Laplace-type Estimator

* Some criterion functions in econometrics is difficult to optimize
* @chernozhukov2003mcmc's *Laplace-type estimator* replaces optimization by simulation

* Let $L_n(\theta)$ be an criterion function (say, OLS, (negative) log likelihood, etc),

$$
f_n (\theta) = \frac{\exp(-L_n(\theta))\pi(\theta)}{\int_{\Theta} \exp(-L_n(\theta))\pi(\theta)}
$$

and $\pi(\theta)$ be the density of a prior.

* The smaller is the value of the objective function, the larger it weighs. 
* Exponential transformation comes from [Laplace approximation](https://en.wikipedia.org/wiki/Laplace's_method).

* Use MCMC to simulate the distribution of $\theta$.
* For a linear regression model, the code block compares the OLS estimator with the LTE estimator 

In [ ]:
library(magrittr)

# DGP
n <- 100
b0 <- c(.1, .1)
X <- cbind(1, rnorm(n))
Y <- X %*% b0 + rnorm(n)

In [ ]:
# Laplace-type estimator
# L <- function(b) -0.5 * sum((Y - X %*% b)^2) - 0.5 * crossprod(b - c(0, 0))
# notice the "minus" sign of the OLS objective function
# here we use a normal prior around (0,0) with var 1

L <- function(b) -0.5*sum((Y - X %*% b)^2) # flat prior

nbatch <- 10000
out <- mcmc::metrop(obj = L, initial = c(0, 0), nbatch = nbatch, nspac = 20)$batch

# summarize the estimation
bhat2 <- out[-(1:round(nbatch / 10)), 2] # remove the burn in
bhat2_point <- mean(bhat2)
bhat2_sd <- sd(bhat2)
bhat2_CI <- quantile(bhat2, c(.025, .975))

# report results
print(cat(
  "The posterior mean =", bhat2_point, " sd = ", bhat2_sd,
  " \n and C.I.=(", bhat2_CI, ")"
))

plot(density(bhat2), main = "posterior from normal prior")

In [ ]:
# OLS estimation

b2_OLS <- (lm(Y ~ -1 + X) %>% summary() %>% coef())[2, ]
# "-1" in lm( ) because X has contained intercept
print(cat(
  "The OLS point est =", b2_OLS[1], " sd = ", b2_OLS[2],
  " \n and C.I.=(", c(b2_OLS[1] - 1.96 * b2_OLS[2], b2_OLS[1] + 1.96 * b2_OLS[2]), ")"
))

## Reading

* [ISLR](https://www.statlearning.com/): Ch 5.2
* [CASI](https://hastie.su.domains/CASI/): Ch.2.1; 3.3--4; 13.4; 10.2-4